<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in New York City</font></h1>

## Introduction

In this lab, we will convert addresses into their equivalent latitude and longitude values. Also, we will use the Foursquare API to explore neighborhoods in both Manhattan and Toronto. We will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. We will use the *k*-means clustering algorithm to complete this task. We will use the Folium library to visualize the neighborhoods in Manhattan and Toronto and their emerging clusters. Finally, we will group the similar neighborhoods in both boroughs.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [325]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Download and Explore Dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the files and placed it on the server, so you can simply run a `wget` command and access the data. So let's go ahead and do that.

#### Load and explore the data

Next, let's load the data.

In [326]:
with open(r"C:\Users\AZZAM\Downloads\newyork_data.json") as json_data:
    newyork_data = json.load(json_data)

Let's take a quick look at the data.

In [327]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [328]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [329]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [330]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [331]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [332]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [333]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [334]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

Let's segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [335]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Let's get the geographical coordinates of Manhattan.

In [336]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.

In [337]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

### Now we add the data of Toronto

In [338]:
# import BeautifulSoup for scraping the Web
from bs4 import BeautifulSoup

In [339]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(wiki_url)
soup = BeautifulSoup(response.text, 'html.parser')

In [340]:
table = soup.find('table', {'class':'wikitable sortable'}).tbody
#print(table)

In [341]:
rows = table.find_all('tr')
columns = [v.text.replace('\n','') for v in rows[0].find_all('th')]
print(columns)

['Postal Code', 'Borough', 'Neighbourhood']


In [342]:
print(rows[0:5])

[<tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>, <tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>, <tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>, <tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>, <tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>]


In [343]:
df = pd.DataFrame(columns=columns)
for i in range(1,len(rows)):
    tds = rows[i].find_all('td')
    
    if len(tds) < 3:
        values = [tds[0].text.replace('\n',''), tds[1].text.replace('\n',''), tds[2].text.replace('\n','')]
    else:
        values = [td.text.replace('\n','') for td in tds]
    df = df.append(pd.Series(values, index=columns), ignore_index=True) 

In [344]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [345]:
# Remove rows with 'Not assigned' in their Borough attribute
df = df[df['Borough'].replace('Not assigned',np.nan).notnull()]

In [346]:
# Reset the index after removing some rows
df = df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [347]:
# If the number of the unique postal codes is equal to postal codes in the original dataframe, then all postal codes are unique
if df.shape[0] == df.groupby(['Postal Code'])['Postal Code'].value_counts().shape[0]:
    print('Every postal code in the data frame is unique')

Every postal code in the data frame is unique


### The dataframe shape (rows, columns) is:

In [348]:
# Show the shape of the cleaned dataframe
print(df.shape)

(103, 3)


### Adding latitudes and longitudes

In [349]:
lat_long_df = pd.read_csv('https://cocl.us/Geospatial_data')
lat_long_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [350]:
# let's take the values of latitude and longitudes from the lat_long_df dataframe
df_new = df.merge(lat_long_df, on='Postal Code', how='left')
df_new.head(30)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [351]:
# let's study Toronto neighborhoods only:
import re

# new dataframe for Toronto 
toronto_df = pd.DataFrame(columns=['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'])


#extract Toronto areas from the table
for i in range(1,len(df_new)):
    if bool(re.search('Toronto',df_new.iloc[i,1])):
        toronto_df =  toronto_df.append(df_new.iloc[i,:])
toronto_df = toronto_df.reset_index(drop=True)

In [352]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


In [353]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [354]:
CLIENT_ID = '5CJGIIETE2SAHBU0N2A4TSSB5C4NAFYONAPWVSYEXN0C0DBD' # your Foursquare ID
CLIENT_SECRET = 'SGJ04C335VVQCUMLW1APLK00ODECNKOR0ZSE0QCGNCIHUSAF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5CJGIIETE2SAHBU0N2A4TSSB5C4NAFYONAPWVSYEXN0C0DBD
CLIENT_SECRET:SGJ04C335VVQCUMLW1APLK00ODECNKOR0ZSE0QCGNCIHUSAF


## 2.A Explore Neighborhoods in Manhattan

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [355]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [356]:
# type your answer here

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )



Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


#### Let's check the size of the resulting dataframe

In [357]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3185, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


Let's check how many venues were returned for each neighborhood

In [358]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,74,74,74,74,74,74
Carnegie Hill,87,87,87,87,87,87
Central Harlem,45,45,45,45,45,45
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,41,41,41,41,41,41
East Village,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [359]:
print('There are {} uniques categories in Manhattan.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 326 uniques categories in Manhattan.


## 2.B Explore Neighborhoods in Toronto

In [360]:
# let's get the Toronto venues and their related info
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [361]:
print(toronto_venues.shape)
toronto_venues.head()

(1638, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Let's check how many venues were returned for each neighborhood

In [362]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,65,65,65,65,65,65
Christie,17,17,17,17,17,17
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


#### Let's find out how many unique categories can be curated from all the returned venue

In [363]:
print('There are {} uniques categories in Toronto.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories in Toronto.


<a id='item3'></a>

In [364]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3185, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [365]:
print(toronto_venues.shape)
toronto_venues.head()

(1638, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [366]:
merged_venues = pd.concat([manhattan_venues, toronto_venues], axis=0)
merged_venues.shape

(4823, 7)

In [367]:
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [368]:
toronto_data = toronto_df.drop('Postal Code', axis=1)
toronto_data.columns = manhattan_data.columns
merged_data = pd.concat([manhattan_data, toronto_data], axis=0)
print((manhattan_data.shape[0] + toronto_data.shape[0]), merged_data.shape[0])

79 79


In [369]:
toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,East Toronto,The Beaches,43.676357,-79.293031


In [370]:
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


## 3. Analyze Each Neighborhood

In [371]:
# one hot encoding
merged_onehot = pd.get_dummies(merged_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
merged_onehot['Neighborhood'] = merged_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [merged_onehot.columns[-1]] + list(merged_onehot.columns[:-1])
merged_onehot = merged_onehot[fixed_columns]

merged_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates 

And let's examine the new dataframe size.

In [372]:
merged_onehot.shape

(4823, 370)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [373]:
merged_grouped = merged_onehot.groupby('Neighborhood').mean().reset_index()
merged_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates 

#### Let's confirm the new size

In [374]:
merged_grouped.shape

(79, 370)

#### Let's print each neighborhood along with the top 5 most common venues

In [375]:
num_top_venues = 5

for hood in merged_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = merged_grouped[merged_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.11
1          Hotel  0.07
2    Coffee Shop  0.05
3            Gym  0.05
4  Shopping Mall  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1        Beer Bar  0.04
2    Cocktail Bar  0.04
3      Restaurant  0.04
4  Farmers Market  0.04


----Brockton, Parkdale Village, Exhibition Place----
               venue  freq
0               Café  0.12
1     Breakfast Spot  0.08
2        Coffee Shop  0.08
3        Yoga Studio  0.04
4  Convenience Store  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.12
1              Garden  0.06
2       Burrito Place  0.06
3                Park  0.06
4       Garden Center  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0  Airport Terminal  0.12
1    Airport Lounge  

                         venue  freq
0               Breakfast Spot  0.14
1                    Gift Shop  0.14
2                  Coffee Shop  0.07
3  Eastern European Restaurant  0.07
4                    Bookstore  0.07


----Queen's Park, Ontario Provincial Government----
         venue  freq
0  Coffee Shop  0.25
1  Yoga Studio  0.03
2     Creperie  0.03
3          Bar  0.03
4     Beer Bar  0.03


----Regent Park, Harbourfront----
         venue  freq
0  Coffee Shop  0.17
1       Bakery  0.06
2         Park  0.06
3          Pub  0.06
4         Café  0.06


----Richmond, Adelaide, King----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.05
2        Hotel  0.04
3          Gym  0.04
4   Restaurant  0.04


----Roosevelt Island----
             venue  freq
0             Park  0.06
1       Playground  0.03
2      Dry Cleaner  0.03
3   Sandwich Place  0.03
4  Bubble Tea Shop  0.03


----Rosedale----
         venue  freq
0         Park  0.50
1   Playground  0.25
2        Trail  

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [376]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [377]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = merged_grouped['Neighborhood']

for ind in np.arange(merged_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(merged_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Coffee Shop,Gym,Memorial Site,Shopping Mall,Playground,Food Court,Boat or Ferry,BBQ Joint
1,Berczy Park,Coffee Shop,Restaurant,Farmers Market,Bakery,Beer Bar,Café,Seafood Restaurant,Cocktail Bar,Cheese Shop,Clothing Store
2,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Convenience Store,Stadium,Burrito Place,Nightclub,Furniture / Home Store,Bar,Bakery
3,"Business reply mail Processing Centre, South C...",Light Rail Station,Pizza Place,Gym / Fitness Center,Fast Food Restaurant,Comic Shop,Brewery,Burrito Place,Smoke Shop,Skate Park,Garden
4,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Terminal,Airport Lounge,Boat or Ferry,Airport Service,Rental Car Location,Coffee Shop,Sculpture Garden,Boutique,Bar,Harbor / Marina


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [378]:
# set number of clusters
kclusters = 10

merged_grouped_clustering = merged_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(merged_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 0, 0, 5, 0, 5, 0, 5, 0, 5])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [379]:
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] =  kmeans.labels_
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Battery Park City,Park,Hotel,Coffee Shop,Gym,Memorial Site,Shopping Mall,Playground,Food Court,Boat or Ferry,BBQ Joint,5
1,Berczy Park,Coffee Shop,Restaurant,Farmers Market,Bakery,Beer Bar,Café,Seafood Restaurant,Cocktail Bar,Cheese Shop,Clothing Store,0
2,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Convenience Store,Stadium,Burrito Place,Nightclub,Furniture / Home Store,Bar,Bakery,0
3,"Business reply mail Processing Centre, South C...",Light Rail Station,Pizza Place,Gym / Fitness Center,Fast Food Restaurant,Comic Shop,Brewery,Burrito Place,Smoke Shop,Skate Park,Garden,5
4,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Terminal,Airport Lounge,Boat or Ferry,Airport Service,Rental Car Location,Coffee Shop,Sculpture Garden,Boutique,Bar,Harbor / Marina,0


In [380]:
merged_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [381]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Battery Park City,Park,Hotel,Coffee Shop,Gym,Memorial Site,Shopping Mall,Playground,Food Court,Boat or Ferry,BBQ Joint,5
1,Berczy Park,Coffee Shop,Restaurant,Farmers Market,Bakery,Beer Bar,Café,Seafood Restaurant,Cocktail Bar,Cheese Shop,Clothing Store,0
2,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Convenience Store,Stadium,Burrito Place,Nightclub,Furniture / Home Store,Bar,Bakery,0
3,"Business reply mail Processing Centre, South C...",Light Rail Station,Pizza Place,Gym / Fitness Center,Fast Food Restaurant,Comic Shop,Brewery,Burrito Place,Smoke Shop,Skate Park,Garden,5
4,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Terminal,Airport Lounge,Boat or Ferry,Airport Service,Rental Car Location,Coffee Shop,Sculpture Garden,Boutique,Bar,Harbor / Marina,0


In [382]:
merged_merged = merged_data
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged_merged = merged_merged.merge(neighborhoods_venues_sorted, how='inner')

merged_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Manhattan,Marble Hill,40.876551,-73.910660,Coffee Shop,Discount Store,Gym,Yoga Studio,Donut Shop,Supplement Shop,Kids Store,Seafood Restaurant,Tennis Stadium,Steakhouse,0
1,Manhattan,Chinatown,40.715618,-73.994279,Chinese Restaurant,Bakery,Cocktail Bar,Vietnamese Restaurant,Dessert Shop,Spa,American Restaurant,Salon / Barbershop,Optical Shop,Noodle House,5
2,Manhattan,Washington Heights,40.851903,-73.936900,Café,Bakery,Chinese Restaurant,Mobile Phone Shop,Sandwich Place,Tapas Restaurant,Coffee Shop,Park,New American Restaurant,Deli / Bodega,8
3,Manhattan,Inwood,40.867684,-73.921210,Café,Mexican Restaurant,Lounge,Restaurant,Bakery,Park,Deli / Bodega,American Restaurant,Chinese Restaurant,Caribbean Restaurant,8
4,Manhattan,Hamilton Heights,40.823604,-73.949688,Pizza Place,Deli / Bodega,Café,Coffee Shop,Mexican Restaurant,Bakery,School,Sandwich Place,Caribbean Restaurant,Chinese Restaurant,8


In [383]:
merged_merged.shape

(79, 15)

Finally, let's visualize the resulting clusters

In [384]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged_merged['Latitude'], merged_merged['Longitude'], merged_merged['Neighborhood'], merged_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

In [385]:
merged_grouped = merged_merged.groupby('Cluster Labels')

#### Cluster 1

In [386]:
merged_merged.loc[merged_merged['Cluster Labels'] == 0, merged_merged.columns[[1] + list(range(5, merged_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Marble Hill,Discount Store,Gym,Yoga Studio,Donut Shop,Supplement Shop,Kids Store,Seafood Restaurant,Tennis Stadium,Steakhouse,0
5,Manhattanville,Mexican Restaurant,Bar,Seafood Restaurant,Italian Restaurant,Sushi Restaurant,Spanish Restaurant,Supermarket,Climbing Gym,Bakery,0
17,Chelsea,Art Gallery,American Restaurant,Ice Cream Shop,Hotel,Bakery,Italian Restaurant,Seafood Restaurant,Cycle Studio,Market,0
29,Financial District,Pizza Place,Cocktail Bar,Café,Gym / Fitness Center,Salad Place,Steakhouse,Hotel,Italian Restaurant,Bar,0
37,Stuyvesant Town,Boat or Ferry,Bar,Bistro,Harbor / Marina,Coffee Shop,Cocktail Bar,Farmers Market,Fountain,Baseball Field,0
40,"Regent Park, Harbourfront",Park,Pub,Bakery,Café,Breakfast Spot,Theater,Restaurant,Bank,Dessert Shop,0
41,"Queen's Park, Ontario Provincial Government",Yoga Studio,Italian Restaurant,Fried Chicken Joint,Sandwich Place,Bar,Bank,Distribution Center,Beer Bar,Diner,0
42,"Garden District, Ryerson",Coffee Shop,Café,Cosmetics Shop,Japanese Restaurant,Bubble Tea Shop,Diner,Hotel,Ramen Restaurant,Theater,0
43,St. James Town,Café,Cocktail Bar,Cosmetics Shop,Restaurant,American Restaurant,Seafood Restaurant,Lingerie Store,Gastropub,Moroccan Restaurant,0
45,Berczy Park,Restaurant,Farmers Market,Bakery,Beer Bar,Café,Seafood Restaurant,Cocktail Bar,Cheese Shop,Clothing Store,0


In [387]:
# similar nighborhoods in Manhattan and Toronto
merged_grouped.get_group(0)[['Neighborhood', 'Borough']]

,Neighborhood,Borough
0,Marble Hill,Manhattan
5,Manhattanville,Manhattan
17,Chelsea,Manhattan
29,Financial District,Manhattan
37,Stuyvesant Town,Manhattan
40,"Regent Park, Harbourfront",Downtown Toronto
41,"Queen's Park, Ontario Provincial Government",Downtown Toronto
42,"Garden District, Ryerson",Downtown Toronto
43,St. James Town,Downtown Toronto
45,Berczy Park,Downtown Toronto


#### Cluster 2

In [388]:
merged_merged.loc[merged_merged['Cluster Labels'] == 1, merged_merged.columns[[1] + list(range(4, merged_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
61,"Forest Hill North & West, Forest Hill Road Park",Park,Trail,Sushi Restaurant,Jewelry Store,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,1


In [389]:
# similar nighborhoods in Manhattan and Toronto
merged_grouped.get_group(1)[['Neighborhood', 'Borough']]

,Neighborhood,Borough
61,"Forest Hill North & West, Forest Hill Road Park",Central Toronto


#### Cluster 3

In [390]:
merged_merged.loc[merged_merged['Cluster Labels'] == 2, merged_merged.columns[[1] + list(range(4, merged_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
58,Lawrence Park,Park,Bus Line,Swim School,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,2


In [391]:
# similar nighborhoods in Manhattan and Toronto
merged_grouped.get_group(2)[['Neighborhood', 'Borough']]

,Neighborhood,Borough
58,Lawrence Park,Central Toronto


#### Cluster 4

In [392]:
merged_merged.loc[merged_merged['Cluster Labels'] == 3, merged_merged.columns[[1] + list(range(4, merged_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
59,Roselawn,Garden,Women's Store,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,3


In [393]:
# similar nighborhoods in Manhattan and Toronto
merged_grouped.get_group(3)[['Neighborhood', 'Borough']]

,Neighborhood,Borough
59,Roselawn,Central Toronto


#### Cluster 5

In [394]:
merged_merged.loc[merged_merged['Cluster Labels'] == 4, merged_merged.columns[[1] + list(range(4, merged_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
69,"Moore Park, Summerhill East",Summer Camp,Trail,Women's Store,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,4


In [395]:
# similar nighborhoods in Manhattan and Toronto
merged_grouped.get_group(4)[['Neighborhood', 'Borough']]

,Neighborhood,Borough
69,"Moore Park, Summerhill East",Central Toronto


#### Cluster 6

In [396]:
merged_merged.loc[merged_merged['Cluster Labels'] == 5, merged_merged.columns[[1] + list(range(4, merged_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
1,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Vietnamese Restaurant,Dessert Shop,Spa,American Restaurant,Salon / Barbershop,Optical Shop,Noodle House,5
6,Central Harlem,African Restaurant,Chinese Restaurant,Bar,Cosmetics Shop,Gym / Fitness Center,American Restaurant,Seafood Restaurant,French Restaurant,Music Venue,Caribbean Restaurant,5
8,Upper East Side,Italian Restaurant,Bakery,Coffee Shop,French Restaurant,Gym / Fitness Center,Yoga Studio,Exhibit,Hotel,Juice Bar,Spa,5
9,Yorkville,Italian Restaurant,Coffee Shop,Gym,Bar,Sushi Restaurant,Pizza Place,Deli / Bodega,Mexican Restaurant,Japanese Restaurant,Diner,5
10,Lenox Hill,Coffee Shop,Italian Restaurant,Pizza Place,Sushi Restaurant,Cocktail Bar,Café,Gym,Gym / Fitness Center,Burger Joint,Salad Place,5
11,Roosevelt Island,Park,Plaza,Japanese Restaurant,Deli / Bodega,Kosher Restaurant,Baseball Field,Sandwich Place,Scenic Lookout,School,Supermarket,5
12,Upper West Side,Italian Restaurant,Coffee Shop,Wine Bar,Bar,Indian Restaurant,Bakery,Café,Pizza Place,Dessert Shop,Middle Eastern Restaurant,5
13,Lincoln Square,Plaza,Café,Concert Hall,Italian Restaurant,Theater,Performing Arts Venue,American Restaurant,Wine Shop,Clothing Store,Gym / Fitness Center,5
14,Clinton,Theater,Gym / Fitness Center,Gym,Sandwich Place,American Restaurant,Coffee Shop,Italian Restaurant,Spa,Hotel,Wine Shop,5
15,Midtown,Coffee Shop,Clothing Store,Bakery,Hotel,Theater,American Restaurant,Sandwich Place,Sporting Goods Shop,Bookstore,Steakhouse,5


In [397]:
# similar nighborhoods in Manhattan and Toronto
merged_grouped.get_group(5)[['Neighborhood', 'Borough']]

,Neighborhood,Borough
1,Chinatown,Manhattan
6,Central Harlem,Manhattan
8,Upper East Side,Manhattan
9,Yorkville,Manhattan
10,Lenox Hill,Manhattan
11,Roosevelt Island,Manhattan
12,Upper West Side,Manhattan
13,Lincoln Square,Manhattan
14,Clinton,Manhattan
15,Midtown,Manhattan


#### Cluster 7

In [398]:
merged_merged.loc[merged_merged['Cluster Labels'] == 6, merged_merged.columns[[1] + list(range(4, merged_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
73,Rosedale,Park,Playground,Trail,Women's Store,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,6


In [399]:
# similar nighborhoods in Manhattan and Toronto
merged_grouped.get_group(6)[['Neighborhood', 'Borough']]

,Neighborhood,Borough
73,Rosedale,Downtown Toronto


#### Cluster 8

In [400]:
merged_merged.loc[merged_merged['Cluster Labels'] == 7, merged_merged.columns[[1] + list(range(4, merged_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
44,The Beaches,Pub,Asian Restaurant,Trail,Health Food Store,Women's Store,Eastern European Restaurant,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,7


In [401]:
# similar nighborhoods in Manhattan and Toronto
merged_grouped.get_group(7)[['Neighborhood', 'Borough']]

,Neighborhood,Borough
44,The Beaches,East Toronto


#### Cluster 9

In [402]:
merged_merged.loc[merged_merged['Cluster Labels'] == 8, merged_merged.columns[[1] + list(range(4, merged_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
2,Washington Heights,Café,Bakery,Chinese Restaurant,Mobile Phone Shop,Sandwich Place,Tapas Restaurant,Coffee Shop,Park,New American Restaurant,Deli / Bodega,8
3,Inwood,Café,Mexican Restaurant,Lounge,Restaurant,Bakery,Park,Deli / Bodega,American Restaurant,Chinese Restaurant,Caribbean Restaurant,8
4,Hamilton Heights,Pizza Place,Deli / Bodega,Café,Coffee Shop,Mexican Restaurant,Bakery,School,Sandwich Place,Caribbean Restaurant,Chinese Restaurant,8
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Sandwich Place,Latin American Restaurant,Deli / Bodega,Steakhouse,Donut Shop,Restaurant,Gas Station,8
25,Manhattan Valley,Bar,Mexican Restaurant,Pizza Place,Park,Coffee Shop,Yoga Studio,Bubble Tea Shop,Latin American Restaurant,Furniture / Home Store,Fried Chicken Joint,8
36,Tudor City,Mexican Restaurant,Café,Park,Deli / Bodega,Coffee Shop,Diner,Sushi Restaurant,Vietnamese Restaurant,Dog Run,Seafood Restaurant,8
47,Christie,Grocery Store,Café,Park,Restaurant,Italian Restaurant,Baby Store,Diner,Athletics & Sports,Coffee Shop,Nightclub,8
49,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Music Venue,Bank,Bar,Café,Furniture / Home Store,Bus Stop,Grocery Store,Park,8
62,"High Park, The Junction South",Mexican Restaurant,Thai Restaurant,Café,Speakeasy,Flea Market,Music Venue,Fried Chicken Joint,Furniture / Home Store,Arts & Crafts Store,Cajun / Creole Restaurant,8
64,"The Annex, North Midtown, Yorkville",Sandwich Place,Café,Coffee Shop,Middle Eastern Restaurant,Vegetarian / Vegan Restaurant,Pub,Pharmacy,Park,Donut Shop,Burger Joint,8


In [403]:
# similar nighborhoods in Manhattan and Toronto
merged_grouped.get_group(8)[['Neighborhood', 'Borough']]

,Neighborhood,Borough
2,Washington Heights,Manhattan
3,Inwood,Manhattan
4,Hamilton Heights,Manhattan
7,East Harlem,Manhattan
25,Manhattan Valley,Manhattan
36,Tudor City,Manhattan
47,Christie,Downtown Toronto
49,"Dufferin, Dovercourt Village",West Toronto
62,"High Park, The Junction South",West Toronto
64,"The Annex, North Midtown, Yorkville",Central Toronto


#### Cluster 10

In [404]:
merged_merged.loc[merged_merged['Cluster Labels'] == 9, merged_merged.columns[[1] + list(range(4, merged_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
60,Davisville North,Hotel,Pizza Place,Food & Drink Shop,Park,Breakfast Spot,Sandwich Place,Gym / Fitness Center,Department Store,Duty-free Shop,Doner Restaurant,9


In [405]:
# similar nighborhoods in Manhattan and Toronto
merged_grouped.get_group(9)[['Neighborhood', 'Borough']]

,Neighborhood,Borough
60,Davisville North,Central Toronto
